# Deployment script with VPC Config

In [1]:
import boto3
import sagemaker
import time
from time import gmtime, strftime
import json
import numpy as np

session = sagemaker.Session()
sm_client = boto3.client('sagemaker')
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# vpc configuration with multiple subnets
vpc_config = {
   'Subnets': [
       'subnet-08ab4f85521900ccb',
       'subnet-00ab8c58bb53cfbc9'
   ],
   'SecurityGroupIds': ['sg-061f2bba40ed5c109']
}

In [3]:
model_package_arn = "arn:aws:sagemaker:us-east-1:515966515324:model-package/mnist-classification-models/2"


In [4]:
print("Approving model...")
sm_client.update_model_package(
   ModelPackageArn=model_package_arn,
   ModelApprovalStatus='Approved'
)

Approving model...


{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:515966515324:model-package/mnist-classification-models/2',
 'ResponseMetadata': {'RequestId': '6fe4647b-46d3-4ce9-a016-cd3b4eacdbb0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6fe4647b-46d3-4ce9-a016-cd3b4eacdbb0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '106',
   'date': 'Wed, 27 Nov 2024 05:24:20 GMT'},
  'RetryAttempts': 0}}

In [5]:
response = sm_client.describe_model_package(
   ModelPackageName=model_package_arn
)

print("\nUpdated model status:")
print(f"Status: {response['ModelPackageStatus']}")
print(f"Approval Status: {response['ModelApprovalStatus']}")



Updated model status:
Status: Completed
Approval Status: Approved


In [6]:
print("\nCreating endpoint...")
timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_name = f'mnist-endpoint-config-{timestamp}'
endpoint_name = f'mnist-endpoint-{timestamp}'
model_name = f'mnist-model-{timestamp}'



Creating endpoint...


In [7]:
print("Creating model in SageMaker...")
sm_client.create_model(
   ModelName=model_name,
   ExecutionRoleArn=role,
   PrimaryContainer={
       'ModelPackageName': model_package_arn
   },
   VpcConfig=vpc_config,
   EnableNetworkIsolation=True
)

Creating model in SageMaker...


{'ModelArn': 'arn:aws:sagemaker:us-east-1:515966515324:model/mnist-model-2024-11-27-05-24-25',
 'ResponseMetadata': {'RequestId': '4542d48a-cb23-4dc7-9620-7179db6c0664',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4542d48a-cb23-4dc7-9620-7179db6c0664',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Wed, 27 Nov 2024 05:24:26 GMT'},
  'RetryAttempts': 0}}

In [8]:
print("Creating endpoint configuration...")
sm_client.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[{
       'InstanceType': 'ml.m5.large',
       'InitialInstanceCount': 1,
       'InitialVariantWeight': 1,
       'ModelName': model_name,
       'VariantName': 'AllTraffic'
   }]
)

Creating endpoint configuration...


{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:515966515324:endpoint-config/mnist-endpoint-config-2024-11-27-05-24-25',
 'ResponseMetadata': {'RequestId': 'a2acacd3-e182-4826-97fe-5351563a3f04',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a2acacd3-e182-4826-97fe-5351563a3f04',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '122',
   'date': 'Wed, 27 Nov 2024 05:24:35 GMT'},
  'RetryAttempts': 0}}

In [9]:
print("Creating endpoint...")
sm_client.create_endpoint(
   EndpointName=endpoint_name,
   EndpointConfigName=endpoint_config_name
)

print(f"\nCreating endpoint: {endpoint_name}")
print("The endpoint may take a few minutes to be ready...")

Creating endpoint...

Creating endpoint: mnist-endpoint-2024-11-27-05-24-25
The endpoint may take a few minutes to be ready...


In [10]:
#verify status
print("Waiting for endpoint to be ready...")
while True:
   endpoint_desc = sm_client.describe_endpoint(EndpointName=endpoint_name)
   status = endpoint_desc['EndpointStatus']
   print(f"Status: {status}")
   
   if status == 'InService':
       print("Endpoint created successfully!")
       break
   elif status == 'Failed':
       failure_reason = endpoint_desc.get('FailureReason', 'Unknown reason')
       print(f"Endpoint creation failed. Reason: {failure_reason}")
       
       #endpoint logs
       print("\nEndpoint logs:")
       logs_client = boto3.client('logs')
       log_group = f"/aws/sagemaker/Endpoints/{endpoint_name}"
       
       try:
           log_streams = logs_client.describe_log_streams(
               logGroupName=log_group,
               orderBy='LastEventTime',
               descending=True,
               limit=1
           )
           
           if log_streams.get('logStreams'):
               stream_name = log_streams['logStreams'][0]['logStreamName']
               logs = logs_client.get_log_events(
                   logGroupName=log_group,
                   logStreamName=stream_name
               )
               
               for event in logs['events']:
                   print(event['message'])
       except Exception as e:
           print(f"Could not fetch logs: {str(e)}")
           
       raise Exception(f"Endpoint creation failed: {failure_reason}")
       
   time.sleep(30)

Waiting for endpoint to be ready...
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Endpoint created successfully!


In [11]:
#test prediction
test_images = np.load('processed_data/test_images.npy')
test_labels = np.load('processed_data/test_labels.npy')

In [12]:
test_image = test_images[0:1].tolist()  # take first image

In [13]:
runtime = boto3.client('sagemaker-runtime')

print("\nMaking prediction...")
response = runtime.invoke_endpoint(
   EndpointName=endpoint_name,
   ContentType='application/json',
   Body=json.dumps(test_image)
)


Making prediction...


In [14]:
prediction = json.loads(response['Body'].read().decode())
predicted_label = np.argmax(prediction)
actual_label = test_labels[0]

print(f"\nActual label: {actual_label}")
print(f"Predicted label: {predicted_label}")


Actual label: 7
Predicted label: 0
